In [1]:
import tensorflow as tf
import os
import _pickle as cPickle
import numpy as np

CIFAR_DIR = "C:\\Users\\i076453\\Downloads\\tech\\ml\\cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
def load_data(filename):
    """ read data from data file """
    with open(filename, 'rb') as f:
        data = cPickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

# tensorflow.dataset
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)     # 不做filter, 将所有的数据都加载
            all_labels.append(labels)
        self._data = np.vstack(all_data)  # 纵向合并成
        self._data = self._data / 127.5 - 1 # 进行归一化 
            
        self._labels = np.hstack(all_labels) # 横向合并
        
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
        
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch"""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)] 
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

# batch_data, batch_labels = train_data.next_batch(10)
# print(batch_data)
# print(batch_labels)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [3]:
def separable_conv_block(x, ouput_channel_number, name):
    """separable block implementation"""
    """
    Args:
    - x
    - ouput_channel_number eg: output channel 
    - name
    """
    
    with tf.variable_scope(name):
        # 获取通道数目，为了将每个通道都拆开
        input_channel = x.get_shape().as_list()[-1]
        # 调用api 拆分通道 是一个列表： [channel1, channel2, ,,,,]
        channel_wise_x = tf.split(
                        x,
                        input_channel,
                        axis = 3 # 第四个维度的index
                        )
        # 在每个channel做卷积
        # 需要一个列表接输出
        output_channels = []
        for i in range(len(channel_wise_x)):
            output_channel = tf.keras.layers.Conv2D(
                                            1,
                                            (3,3),
                                            strides = (1,1),
                                            padding = 'same',
                                            activation = tf.nn.relu,
                                            name = 'conv%d' % i
                                            )(channel_wise_x[i])
            output_channels.append(output_channel)
        # 拼接这些wise channel卷积
        concat_layer = tf.concat(output_channels, axis = 3)
        
        # 再经过一个1x1的卷积
        conv1_1 = tf.keras.layers.Conv2D(
                                    ouput_channel_number,
                                    (1,1),
                                    strides = (1,1),
                                    padding = 'same',
                                    activation = tf.nn.relu,
                                    name = 'conv1_1')(concat_layer)
        
        return conv1_1

x = tf.placeholder(tf.float32, [None, 3072]) # data 的placeholder, None表示不确定的
# y的shape是 [0.6,5,3]
y = tf.placeholder(tf.int64, [None]) # labels的placeholder

# 转换成多通道
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 交换通道
# 32 x 32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1]) # [-1, 32, 32, 3]


# conv1 : 神经元图，feature_map, 输出图像
conv1 = tf.keras.layers.Conv2D(
                        32,     # output channel number
                        (3,3),  # kernel size
                        padding = 'same',  # 表示输出输入图像大小一致， 还有一个值是valid
                        activation=tf.nn.relu, # 激活函数
                        name = 'conv1' # 名字
                        )(x_image)

# 构建池化层
# 16 x 16 就是将原来的图片大小降为原来的1/2
pooling1 = tf.keras.layers.MaxPool2D(
                                  (2,2), # kernel size
                                  (2,2), # stride 步长 
                                  name = 'pooling1')(conv1)

# 添加两个separable层
separable_2a = separable_conv_block(pooling1, 32, name = "separable_2a")
separable_2b = separable_conv_block(separable_2a, 32, name = "separable_2b")

# 构建池化层
# 16 x 16 就是将原来的图片大小降为原来的1/2
pooling2 = tf.keras.layers.MaxPool2D(
                                  (2,2), # kernel size
                                  (2,2), # stride 步长 
                                  name = 'pooling2')(separable_2a)


# 添加两个separable层
separable_3a = separable_conv_block(pooling2, 32, name = "separable_3a")
separable_3b = separable_conv_block(separable_3a, 32, name = "separable_3b")

# 构建池化层
# 16 x 16 就是将原来的图片大小降为原来的1/2
pooling3 = tf.keras.layers.MaxPool2D(
                                  (2,2), # kernel size
                                  (2,2), # stride 步长 
                                  name = 'pooling3')(separable_3b)

# 展平图片 [None, 4 x 4 x 32]
flatten = tf.keras.layers.Flatten()(pooling3)
    
# 构建全连接层
y_ = tf.keras.layers.Dense(10)(flatten)

# 交叉熵
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# 它做了下面三件事
# y_ -> softmax
# y -> one hot
# loss = ylogy_

# bool 
predict = tf.argmax(y_, 1) # 在第二个维度求最大值

# [1,0,0,1,0,0,1,1...]
correct_predict = tf.equal(predict, y)

# 准确率
accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float64))

# 有了loss目标函数和准确率后，再去做一个
# AdamOptimizer 是一个剔除向量的变种，1e-3就是10的-3次方0.001 
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [4]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
#  run 10000, acc: 0.60000
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict = {
                x: batch_data,
                y: batch_labels
            }
        )
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' \
                % (i+1, loss_val, acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size) 
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels
                    }
                ) 
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' % (i+1, test_acc))

[Train] Step: 500, loss: 1.52486, acc: 0.45000
[Train] Step: 1000, loss: 1.71060, acc: 0.40000


KeyboardInterrupt: 